Just a reminder:
RCFD2107: Obligations (other than securities and leases) of states and political subdivisions in the
U.S (Consolidated Bank). Dollar amounts in thousands.
RCON2107: Obligations (other than securities and leases) of states and political subdivisions in the
U.S (Domestic Offices). Dollar amounts in thousands.

In [5]:
import pandas as pd
import numpy as np
import zipfile
import io
import os
import gc

directory = os.fsencode('data')
muni_banks_table = pd.DataFrame()
quarter = {'0331': 'Q1', '0630': 'Q2', '0930': 'Q3', '1231': 'Q4'}

for file in os.listdir(directory):
    # prep the zipfiles to open
    filename = os.fsdecode(file)
    zipf = zipfile.ZipFile('data/'+filename)
    
    # find some designation to find the right file:
    words = filename.split()
    call_report_date = words[-1].replace('.zip', '')
    
    # find the bank designation
    bank_filename = 'FFIEC CDR Call Bulk POR {}.txt'.format(call_report_date)
    bank_file = pd.read_csv(zipf.open(bank_filename), sep='\t').set_index('IDRSSD')

    # grab the call schedule
    securities_filename = 'FFIEC CDR Call Schedule RCCI {}.txt'.format(call_report_date)
    securities_file = pd.read_csv(zipf.open(securities_filename), sep='\t', header=[0], skiprows=[1]).set_index('IDRSSD')

    # just suss out the muni data
    muni_loans = securities_file['RCFD2107'].dropna().to_frame()
    muni_loans['CallDate'] = call_report_date[-4:]+quarter[call_report_date[0:4]]
    
    # merge the tables together, append to dataframe
    muni_bank = pd.merge(muni_loans, bank_file, how='left', left_index=True, right_index=True)
    if muni_banks_table.empty:
        muni_banks_table = muni_bank
    else:
        muni_banks_table = pd.concat([muni_banks_table, muni_bank])
        
    # garbage clean up
    del [[bank_file, securities_file, muni_loans, muni_bank]]
    gc.collect()
    bank_file = pd.DataFrame()
    securities_file = pd.DataFrame()
    muni_loans = pd.DataFrame()
    muni_bank = pd.DataFrame()

In [8]:
# let's look at the sums
muni_banks_table[muni_banks_table['RCFD2107'] != 0]
muni_total = muni_banks_table.groupby(['CallDate', 'Financial Institution Name'])['RCFD2107'].sum().unstack(level=-2).fillna(0)
temp = muni_total.transpose()
temp['ALL BANKS'] = temp.sum(axis=1)
muni_total = temp.transpose()
muni_total = muni_total/1000
muni_total['sum'] = muni_total.sum(axis=1)
muni_total = muni_total[muni_total['sum'] != 0]
muni_total.drop('sum', axis=1, inplace=True)

# let's drop the total for now DELETE THIS LINE IF YOU WANT TO KEEP IT
muni_total = muni_total.drop('ALL BANKS', axis=0)

Then taking all that work and converting it into an easy to read csv for heroku to fly with.

In [9]:
muni_total.to_csv('munidata.csv')

In [21]:
import pandas as pd
import numpy as np

df = pd.read_csv('munidata.csv').set_index('Financial Institution Name')
{np.where(df.columns.values == i)[0][0]: i for i in df.columns.values}

df.sort_values(by=['2001Q1'], ascending=True, inplace=True)
    #subset = munidata.head(50).copy()
df

,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,2002Q3,2002Q4,2003Q1,2003Q2,...,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4
Financial Institution Name,,,,,,,,,,,,,,,,,,,,,
"BANK OF AMERICA, NATIONAL ASSOCIATION",3772.000,3606.000,4037.000,3875.000,2427.000,2267.000,2278.000,2476.000,2834.000,3254.000,...,19008.000,19254.000,19411.000,19165.000,19358.000,19710.000,19806.000,18994.000,18882.000,21050.000
FIRST UNION NATIONAL BANK,1775.000,1636.000,1617.000,1684.000,1670.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
SUNTRUST BANK,1112.580,1096.091,1160.797,1198.992,1226.885,1258.677,1316.741,1425.540,1497.262,1698.262,...,3728.800,3562.761,3611.744,3569.159,3545.044,3456.696,3473.553,3450.372,3438.806,3669.387
U.S. BANK NATIONAL ASSOCIATION,964.898,973.536,1436.737,1302.937,1287.519,1412.770,1628.693,1427.320,1508.744,1601.330,...,11654.657,11915.645,12306.237,12156.150,12173.489,12383.267,12136.023,11857.838,11475.295,10854.490
"PNC BANK, NATIONAL ASSOCIATION",559.798,539.683,512.155,488.985,506.951,496.056,479.821,427.106,453.257,407.375,...,12578.392,12736.243,12910.531,13289.869,12930.261,13117.022,13098.043,12458.535,11820.501,11078.792
KEYBANK NATIONAL ASSOCIATION,382.784,399.402,427.891,436.522,474.388,380.036,406.174,370.632,385.314,369.648,...,1165.579,1868.216,2033.599,1726.562,1870.839,2256.318,2255.145,2048.064,2102.959,2017.287
"WACHOVIA BANK, NATIONAL ASSOCIATION",360.929,358.983,376.199,362.202,355.908,2001.000,1979.000,1878.000,1909.000,1925.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
MANUFACTURERS AND TRADERS TRUST COMPANY,354.213,350.903,354.548,355.444,299.684,292.420,380.141,373.113,387.258,488.039,...,136.903,133.752,151.324,137.554,83.071,102.366,112.133,107.350,99.030,107.682
"FIRSTAR BANK, NATIONAL ASSOCIATION",344.345,324.334,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
